<a href="https://colab.research.google.com/github/olonok69/LLM_Notebooks/blob/main/summarization/Langchain_splitting_GTR-T5_large_Document_classification_no_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq transformers -U --progress-bar off
!pip install sentence-transformers  langchain langchain-community langchain-huggingface trl datasets pypdf  -qqq --progress-bar off
! pip install torch torchvision -qqq --progress-bar off

In [3]:
! pip install -qU "langchain-chroma>=0.1.2"

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 101.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 92.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 187.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 133.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.28.3 which is incompatible.
tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have protobuf 5.28.3 which is incompatible.


In [5]:
from langchain.text_splitter import SentenceTransformersTokenTextSplitter
from langchain.document_loaders import DirectoryLoader, TextLoader

# Loaders
from langchain.schema import Document

# Splitters
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Model


# Embedding Support



# Summarizer we'll use for Map Reduce
from langchain.chains.summarize import load_summarize_chain

# Data Science
import numpy as np
from sklearn.cluster import KMeans


from google.colab import userdata
import logging

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import requests

response = requests.get("https://www.gutenberg.org/cache/epub/64317/pg64317.txt")

In [8]:
book_complete_text = response.text

In [9]:
book_complete_text = book_complete_text[5:]

In [10]:
len(book_complete_text)

296854

In [11]:
file_path = "/content/drive/MyDrive/data/book.txt"

In [12]:
with open(file_path, "w", encoding="utf-8") as f:
    f.write(book_complete_text)

In [13]:

with open(file_path, "r",  encoding="utf-8") as f:
    text = f.read()

In [14]:
text = text.replace('\t', ' ')

In [15]:
len(text)

290072

# Text Splitters

- https://python.langchain.com/docs/modules/data_connection/document_transformers/
- https://python.langchain.com/docs/modules/data_connection/document_transformers/recursive_text_splitter

In [16]:
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", "\t"], chunk_size=5000, chunk_overlap=300)



In [17]:
docs = text_splitter.create_documents([text])
num_documents = len(docs)

print (f"Now our book is split up into {num_documents} documents")

Now our book is split up into 63 documents


In [18]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

In [19]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [20]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

In [21]:
path_class = "/content/drive/MyDrive/data/classes.csv"

In [22]:
import os
from typing import List
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import CSVLoader
import logging


def create_vector_db(class_file, db_path, embeddings, metadata_columns):
    """

    Args:
        class_file (_type_): _description_
        chroma_path (_type_): _description_
        embeddings (_type_): _description_
        metadata_columns (_type_): _description_

    Returns:
        _type_: _description_
    """
    loader = CSVLoader(class_file, metadata_columns=metadata_columns)
    documents = loader.load()
    os.makedirs(db_path, exist_ok=True)
    # delete collection if this exists
    db = Chroma.from_documents(
        documents=documents,
        # Chose the embedding you want to use
        embedding=embeddings,
        collection_metadata={"hnsw:space": "cosine"},
    )

    return db


def get_or_create_vector_db(
    class_file: str,
    db_path: str,
    embeddings,
    metadata_columns: List = ["classifier"],
):
    """
    Create Vector Database or read it from persistance location
    """

    logging.info(f"DB VectorStore Path: {db_path}")
    db = create_vector_db(class_file, db_path, embeddings, metadata_columns)

    return db

In [23]:
db = get_or_create_vector_db(
    class_file=path_class, db_path="/content/drive/MyDrive/chroma/test", embeddings=embeddings
)

In [34]:
results = {}

for i, doc in enumerate(docs):
    res = db.similarity_search_with_score(doc.page_content, k=5)
    output = []
    for doc_result, score in res:
        output.append({doc_result.metadata.get("classifier"): score})
    results[i] = output



In [40]:
final_output = {}
for key, value in results.items():
    for item in value:
        key2 = list(item.keys())[0]
        value2 = list(item.values())[0]
        if key2 in final_output:
            final_output[key2] = (final_output[key2] + value2) /2
        else:
            final_output[key2] = value2


In [41]:
final_output

{'Gifts and Entertainment': 0.8722820763583325,
 'Legal Affairs': 0.8663945867746428,
 'Unauthorized Disclosure': 0.8127160912325186,
 'Quotation': 0.9000009447336197,
 'Tax Documents': 0.8981709778308868,
 'Wire Transfer': 0.9143760502338409,
 'Meeting Notes': 0.9128933537285775,
 'Freight Documents': 0.9125692993402481,
 'Stock Manipulation': 0.8734355425990543,
 'Intellectual Property': 0.7336095453938469,
 'Regulatory Collusion': 0.9144457448273897,
 'Healthcare': 0.8453583866357803,
 'Profanity': 0.9144173703778762,
 'Customer Complaints': 0.896117010038858,
 'Statement of Work': 0.7639787793159485,
 'Legal Agreements': 0.7682812247805375,
 'Statement of Accounts': 0.9121315248776227,
 'Sales and Revenue': 0.8113168756244704,
 'Financial Statement': 0.8848685952834785,
 'Corporate Sabotage': 0.8898799545131624,
 'Business Context Preview': 0.7989053428173065,
 'Adult Racy and Gory Images': 0.9035358484834433,
 'Resume': 0.9461051225662231,
 'Bank Statement': 0.9160382151603699,
 '

In [46]:
import heapq

k_keys_sorted = heapq.nlargest(5, final_output, key=final_output.get)
k_keys_sorted

['Resume',
 'Bank Statement',
 'Regulatory Collusion',
 'Profanity',
 'Wire Transfer']

In [44]:
sorted(final_output, key=final_output.get, reverse=True)[:5]

['Resume',
 'Bank Statement',
 'Regulatory Collusion',
 'Profanity',
 'Wire Transfer']

In [31]:
# create output
output = []
for doc
for doc_result, score in res:
    output.append({doc_result.metadata.get("classifier"): score})

In [32]:
output

[{'Gifts and Entertainment': 0.8915141820907593},
 {'Legal Affairs': 0.9060986638069153},
 {'Unauthorized Disclosure': 0.9142679572105408},
 {'Quotation': 0.9246313571929932},
 {'Tax Documents': 0.9251895546913147}]